Battery RUL (Remaining Useful Life) Prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Importing and understanding the battery data set for NASA

In [7]:
import os
import scipy.io

#  Set this to your actual top-level folder
base_path = "/Users/safwanshaikh/Downloads/AML/Battery-RUL-Prediction/Battery_Data_Set"

def extract_discharge_cycles_updated(mat_file_path, key_name):
    data = scipy.io.loadmat(mat_file_path)
    cycles = data[key_name][0][0]['cycle'][0]

    discharge_data = []

    for idx, cycle in enumerate(cycles):
        if cycle['type'][0] != 'discharge':
            continue

        cycle_data = cycle['data']
        voltage = cycle_data['Voltage_measured'][0][0].flatten()
        current = cycle_data['Current_measured'][0][0].flatten()
        temperature = cycle_data['Temperature_measured'][0][0].flatten()
        time_sec = cycle_data['Time'][0][0].flatten()

        try:
            capacity = cycle_data['Capacity'][0][0][0][0]
        except:
            capacity = np.nan

        df = pd.DataFrame({
            'battery_id': key_name,
            'cycle_num': idx + 1,
            'time_sec': time_sec,
            'voltage': voltage,
            'current': current,
            'temperature': temperature,
            'capacity': capacity
        })

        discharge_data.append(df)

    return pd.concat(discharge_data, ignore_index=True) if discharge_data else None

#Recursively load all `.mat` files from subfolders
all_dfs = []

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".mat") and file.startswith("B"):
            mat_path = os.path.join(root, file)
            key_name = file.split('.')[0]
            try:
                df = extract_discharge_cycles_updated(mat_path, key_name)
                if df is not None:
                    all_dfs.append(df)
            except Exception as e:
                print(f"Error processing {file}: {e}")

# 🔗 Merge all into one DataFrame
merged_df = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()


In [8]:
merged_df.shape

(835422, 7)

In [9]:
merged_df.head(10)

,battery_id,cycle_num,time_sec,voltage,current,temperature,capacity
0,B0052,1,0.000,3.839569,0.003881,9.127700,0.860659
1,B0052,1,9.313,3.839317,0.000240,9.109016,0.860659
2,B0052,1,19.406,3.467054,-1.992228,9.069624,0.860659
3,B0052,1,28.797,3.441731,-1.994519,9.091822,0.860659
4,B0052,1,38.141,3.420127,-1.993204,9.126609,0.860659
5,B0052,1,47.563,3.400286,-1.994815,9.175103,0.860659
6,B0052,1,56.953,3.381112,-1.994436,9.215666,0.860659
7,B0052,1,66.235,3.362287,-1.995251,9.279857,0.860659
8,B0052,1,75.625,3.343357,-1.993564,9.388372,0.860659
9,B0052,1,85.000,3.324146,-1.991669,9.453485,0.860659


In [10]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835422 entries, 0 to 835421
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   battery_id   835422 non-null  object 
 1   cycle_num    835422 non-null  int64  
 2   time_sec     835422 non-null  float64
 3   voltage      835422 non-null  float64
 4   current      835422 non-null  float64
 5   temperature  835422 non-null  float64
 6   capacity     830026 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 44.6+ MB


In [11]:
merged_df.describe()

,cycle_num,time_sec,voltage,current,temperature,capacity
count,835422.000000,835422.000000,835422.000000,835422.000000,835422.000000,830026.000000
mean,147.165454,1983.882216,3.422200,-1.536301,27.690855,1.459198
std,141.900422,1472.518834,0.410522,1.211351,13.619596,0.370423
min,1.000000,0.000000,0.192738,-4.038796,2.904756,0.000000
25%,42.000000,825.875000,3.284805,-2.010638,12.387256,1.292408
50%,91.000000,1710.515000,3.469811,-1.988676,30.326697,1.525268
75%,221.000000,2742.312000,3.665822,-0.992897,35.985494,1.759836
max,614.000000,6574.671000,4.542427,0.030461,69.869746,2.640149


In [14]:
merged_df['battery_id'].value_counts()

battery_id
B0007    50285
B0006    50285
B0005    50285
B0033    47065
B0036    47065
B0034    47065
B0018    34866
B0028    32676
B0025    32676
B0027    32676
B0026    32676
B0043    25795
B0044    25795
B0042    25795
B0047    24519
B0048    24519
B0045    24519
B0046    24519
B0054    22665
B0056    22662
B0055    22662
B0040    20201
B0038    20201
B0039    20201
B0041    13519
B0053    11090
B0029     6351
B0032     6351
B0031     6351
B0030     6351
B0050     5934
B0049     5934
B0051     5934
B0052     5934
Name: count, dtype: int64